# Base de datos. Eligiendo la compañía 🕵🏻

En este jupyter se encuentra todo el proceso de limpieza y selección de las oficinas descargadas del dataset de "companies.json" proporcionado por Ironhack.

*Consideraciones*

- Para la selección de las oficinas se tiene en cuenta que la empresa se compone por una plantilla de más o menos 85 empleados por ello la capacidad de la oficina será para 80 - 150 personas. Además puesto que es un sector actual y muy novedoso se exige que el año de fundación de la misma no sea superior al año 2000 y su categoría sea videojuegos.
- Para la elección del país se tendrá en cuenta el análisis previo, que la empresa ya dispone de sedes en China y que su objetivo es la expansión dentro del marco internacional.

## Índice 📎

1. Importación de librerias y funciones
2. Importación de la base de datos "companies"
3. Análisis previo
>- Toma de datos
>- Elección del país
4. Exploración del dataset
5. Selección de oficinas
6. Exportación de los datos
7. Visualización de los datos

## 1. Importación de librerias y funciones 📚

In [1]:
import pandas as pd
import sqlalchemy as alch
import numpy as np
import re
import requests
import seaborn as sns
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from pymongo import MongoClient
from pandas.io.json import json_normalize
import cartoframes
from cartoframes.viz import Map, Layer, popup_element
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster

Otras funciones:

In [4]:
from iteration_utilities import duplicates
from iteration_utilities import unique_everseen
#import Funciones Funciones.py as fu

## 2. Importación de la base de datos "companies" 📖

In [5]:
#Utilizamos el comando MongoClient para establecer la conexión con el servidor
client = MongoClient("localhost:27017")

In [6]:
#Una vez tenemos la conexión establecemos otra conexión a la base de datos con la que vamos a trabajar.
db = client.get_database("Ironhack-lab")
#db.list_collection_names()

In [7]:
#Realizamos lo mismo para collections
companies = db.get_collection("Companies")
companies

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Ironhack-lab'), 'Companies')

## 3. Análisis previo 🤓

*La industria de videojuegos ha experimentado en los últimos años altas tasas de crecimiento, debido al desarrollo de la computación, capacidad de procesamiento e imágenes más reales, generando una recaudación mundial de 57.600 millones de euros durante 2009 y 91.000 millones de dólares en 2016.*

Para realizar este ejercicio contamos con una empresa de videojuegos altamente reconocida y con gran notoriedad de marca en el mercado, por ello a la hora de escoger la ciudad donde se abrirá la nueva sede se tendrá en cuenta los países más competitivos los cuales generan mayores ingresos en el mundo de dicha industria. Para ello se tomará de referencia los datos extraidos de wikipedia: [list of videogames market by country](https://en.wikipedia.org/wiki/List_of_video_games_markets_by_country) mediante la herramienta de webscraping. 

### 3.1 Toma de Datos 🎣

In [8]:
url_paises = "https://en.wikipedia.org/wiki/List_of_video_games_markets_by_country"

In [9]:
html_paises = requests.get(url_paises)

In [10]:
soup = BeautifulSoup(html_paises.content, "html.parser")

Observamos que se encuentra dentro de un tag llamado "table". Abrimos la consola y mediante la siguiente función la filtramos para comprobar si mediante ese filtro podemos seleccionar la información deseada:

In [11]:
#document.querySelectorAll("Table").forEach(elm => elm.style.background = 'red')
table = soup.find("table")

In [12]:
#Reorganización de la tabla
rows = table.find_all("tr")
rows = [row.text.replace("\n\n", ",").replace("\n", ",").replace("\xa0", "").strip(",").split(",") for row in rows]

In [13]:
#Creación del dataframe
nombre_columnas = rows[0]
data = rows[1:]
paises = pd.DataFrame(data)
#paises.head(10)

In [14]:
#Reenombramos las columnas
paises.columns = ["country", "market", "employees", "gamers", "game"]

In [15]:
#Eliminamos las columnas cuya información no es relevante
paises2 = paises.drop(["employees","gamers","game"], axis=1)
paises3= paises2[:11] #Escogemos el top 10 de países

### 3.1 Elección del país 🌎

In [16]:
paises3

,country,market
0,China,$37.9 billion[2]
1,United States,$27.9 billion
2,Japan,$11.5 billion
3,United Kingdom,£5.7 billion
4,Germany,€4.4 billion
5,Canada,$3 billion
6,France,€2.7 billion
7,Australia,$2.9 billion
8,Russia,$2 billion
9,Italy,€1.8 billion


*Observamos que los países dónde se recauda mayor cantidad de dinero son China, Estados Unidos y Japón.* 

Partiendo de este análisis la decisión del CEO será lanzarse al mercado Internacional en Estados Unidos ya que es un mercado bastante competitivo y en el que considera que puede alcanzar una atractiva cuota de mercado además conocen y hablan el mismo idioma.

## 4. Exploración de datos 🔎

A continuación procederemos a realizar una exploración de la base de datos para posteriormente filtrar y proyectar aquellos datos que nos interesan.

In [17]:
#Para hacernos una idea de las keys que contiene nuestro diccionario
companies.find_one().keys()

dict_keys(['_id', 'name', 'permalink', 'crunchbase_url', 'homepage_url', 'blog_url', 'blog_feed_url', 'twitter_username', 'category_code', 'number_of_employees', 'founded_year', 'founded_month', 'founded_day', 'deadpooled_year', 'tag_list', 'alias_list', 'email_address', 'phone_number', 'description', 'created_at', 'updated_at', 'overview', 'image', 'products', 'relationships', 'competitions', 'providerships', 'total_money_raised', 'funding_rounds', 'investments', 'acquisition', 'acquisitions', 'offices', 'milestones', 'video_embeds', 'screenshots', 'external_links', 'partners'])

*Consideraciones: se toma como idea las keys de este diccionario ya que el método find_one de mongo escoge un valor aleatorio, el cual no tiene porque contener las mismas keys que el resto. Partiendo de esta idea:*

Al tratarse de una empresa de videojuegos, el tipo de empresa "category_code" tendrá que ser especifico de esta industria. Para ello visualizamos los datos úncios contenidos en esta variable y seleccionamos el más acorde a nuestro menester en este caso "games_video".

In [18]:
companies.distinct("category_code")

[None,
 'advertising',
 'analytics',
 'automotive',
 'biotech',
 'cleantech',
 'consulting',
 'design',
 'ecommerce',
 'education',
 'enterprise',
 'fashion',
 'finance',
 'games_video',
 'government',
 'hardware',
 'health',
 'hospitality',
 'legal',
 'local',
 'manufacturing',
 'medical',
 'messaging',
 'mobile',
 'music',
 'nanotech',
 'network_hosting',
 'news',
 'nonprofit',
 'other',
 'photo_video',
 'public_relations',
 'real_estate',
 'search',
 'security',
 'semiconductor',
 'social',
 'software',
 'sports',
 'transportation',
 'travel',
 'web']

Los demás requisitos que deberá cumplir la oficina en cuanto a su estructura y tamaño será que se encuentre en Nueva York, Estados Unidos (por los motivos comentados en el análisis previo) y que tenga una capacidad de entre 80 empleados y 150. Además de un año de fundación superior al año 2000.

In [19]:
filtro = {"$and":[
    {"category_code":{"$eq":"games_video"}},
    {"offices.country_code":{"$eq":"USA"}},
    {"offices.city":{"$eq":"New York"}},
    {"number_of_employees":{"$gte": 80, "$lte":150}},
    {"founded_year": {'$gte': 2000}}
]}

Los datos que necesitamos para el posterior análisis son:

In [20]:
proyeccion = {"_id":0, "name":1, "category_code":1, "number_of_employees":1, "offices.city":1, "offices.country_code":1, "offices.latitude":1, "offices.longitude":1, "founded_year":1}

## 5. Selección de oficinas 💼

In [21]:
oficinas = list(companies.find(filtro,proyeccion))

In [22]:
oficinas

[{'name': 'Livestream',
  'category_code': 'games_video',
  'number_of_employees': 120,
  'founded_year': 2007,
  'offices': [{'city': 'New York',
    'country_code': 'USA',
    'latitude': 40.726155,
    'longitude': -73.995625}]},
 {'name': 'Boonty',
  'category_code': 'games_video',
  'number_of_employees': 150,
  'founded_year': 2001,
  'offices': [{'city': 'New York',
    'country_code': 'USA',
    'latitude': 40.717248,
    'longitude': -74.002662}]}]

In [23]:
data = json_normalize(data = oficinas, record_path = "offices", meta = ["name", "category_code", "number_of_employees","founded_year"])

<ipython-input-23-da31137ad14e>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  data = json_normalize(data = oficinas, record_path = "offices", meta = ["name", "category_code", "number_of_employees","founded_year"])


In [24]:
data

,city,country_code,latitude,longitude,name,category_code,number_of_employees,founded_year
0,New York,USA,40.726155,-73.995625,Livestream,games_video,120,2007
1,New York,USA,40.717248,-74.002662,Boonty,games_video,150,2001


## 6. Exportación de los datos 🚀

In [25]:
data.to_csv("oficinas_info.csv")

## 7. Visualización de las oficinas 🗺️

**Livestream**

In [26]:
oficina_mapa = folium.Map(location = [40.726155,-73.995625],zoom_start=25)

In [27]:
livestream = Icon(color = "red",
             prefix = "fa",
             icon = "building",
             icon_color = "white",
             tooltip = "Livestream")

loc = {"location":[40.726155,-73.995625],
      "tooltip": "Livestream"}

marker = Marker(**loc, icon = livestream)

marker.add_to(oficina_mapa)

**Boonty**

In [28]:
boonty = Icon(color = "red",
             prefix = "fa",
             icon = "building-o",
             icon_color = "white",
             tooltip = "Boonty")

loc = {"location":[40.717248,-74.002662],
      "tooltip": "Boonty"}

marker = Marker(**loc, icon = boonty)

marker.add_to(oficina_mapa)

**Representación**

In [29]:
oficina_mapa

⭐ **Tomaremos como LiveStream como la nueva oficina para IronGames**

*Consideraciones*

La distancia entre ambas oficinas es de: 1,15 kilómetros

In [3]:
def distancia(lat1, lon1, lat2, lon2):
    rad=math.pi/180
    dlat=lat2-lat1
    dlon=lon2-lon1
    R=6372.795477598
    a=(math.sin(rad*dlat/2))**2 + math.cos(rad*lat1)*math.cos(rad*lat2)*(math.sin(rad*dlon/2))**2
    distancia=2*R*math.asin(math.sqrt(a))
    return distancia

In [4]:
distancia(40.726155,-73.995625,40.717248,-74.002662)

1.1547098806187885